In [1]:
import json, time
from IPython.display import clear_output
from STservo import ST3215, Calibrator 

In [2]:
class MAPPER:
    def __init__(self):
        with open('Right_Controller.json', 'r') as fr: 
            conf_ctrl = json.load(fp=fr)
        with open('Right_Arm.json'       , 'r') as fr:
            conf_arm = json.load(fp=fr)
        dev_id = [1, 2, 3, 4, 5, 6] 
        for id_int in dev_id:
            id_str = str(id_int)
            conf_ctrl[id_str]['min'] = (conf_ctrl[id_str]['min'] - conf_ctrl[id_str]['corr']) & 0xFF
            conf_arm[ id_str]['min'] = (conf_arm [id_str]['min'] - conf_arm [id_str]['corr']) & 0xFF

        self.conf_ctrl = conf_ctrl
        self.conf_arm  = conf_arm
        self.dev_id    = dev_id

    def getPosiVelo(self, result):
        posi = []
        velo = []
        for id_int in self.dev_id:
            id_str = str(id_int)
            ratio    = (result[id_int]['posi'] - self.conf_ctrl[id_str]['min']) / self.conf_ctrl[id_str]['range']
            posi_arm = self.conf_arm[id_str]['min'] + self.conf_ctrl[id_str]['range'] * ratio
            posi.append(int(posi_arm))
            velo.append(1200) 
        return posi, velo
m = MAPPER()

In [3]:
m.conf_arm, m.conf_ctrl

({'1': {'min': 100, 'range': 2717, 'corr': -1476},
  '2': {'min': 100, 'range': 2383, 'corr': -691},
  '3': {'min': 100, 'range': 2225, 'corr': -249},
  '4': {'min': 100, 'range': 2367, 'corr': -1743},
  '5': {'min': 100, 'range': 3818, 'corr': -1735},
  '6': {'min': 100, 'range': 1474, 'corr': 1896}},
 {'1': {'min': 100, 'range': 2717, 'corr': -1665},
  '2': {'min': 100, 'range': 2388, 'corr': 1346},
  '3': {'min': 100, 'range': 2223, 'corr': 355},
  '4': {'min': 100, 'range': 2323, 'corr': 1506},
  '5': {'min': 100, 'range': 3806, 'corr': 1923},
  '6': {'min': 49, 'range': 1245, 'corr': -2047}})

In [4]:
#!/usr/bin/env python3
"""
ESP32 ST3215 TCP客户端测试程序
通过WiFi控制ESP32上的ST3215舵机
支持所有ST3215类的函数测试
"""

import socket
import json
import time
import sys
from typing import Union, List, Dict, Any, Optional

class ESP32SERVO:
    def __init__(self, host, port=8888):
        self.host = host
        self.port = port
        self.socket = None
        self.connected = False 
    
    def connect(self):
        """连接到ESP32 TCP服务器"""
        try:
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.socket.connect((self.host, self.port))
            self.connected = True
            print(f"成功连接到 {self.host}:{self.port}")
            return True
        except Exception as e:
            print(f"连接失败: {e}")
            return False
    
    def disconnect(self):
        """断开连接"""
        if self.socket:
            self.socket.close()
            self.connected = False
    
    def send_command(self, func: str, **kwargs) -> Optional[Dict[str, Any]]:
        """发送命令到ESP32"""
        if not self.connected:
            print("未连接到服务器")
            return None
        
        command = {"func": func}
        command.update(kwargs)
        
        command_str = json.dumps(command) + '\n'
        self.socket.send(command_str.encode())
        
        if 'silent' in command and command['silent']: 
            return None 
            
        #self.socket.settimeout(5.0)
        response = self.socket.recv(1024) 
        response = response.decode().strip()
        if not response:
            print("DEBUG: Received empty response")
            return None 
        result = json.loads(response) 
        return result
            
    def getIntFromBytes(self, b):
        return b[0] + (b[1]<<8)
    def getBytesFromInt(self, i):
        return [i & 0xFF, (i >> 8) & 0xFF] 

In [5]:
esp32_ip = "192.168.3.132"
servo_remote = ESP32SERVO(esp32_ip)
print(f"连接到ESP32 ({esp32_ip}:8888)...")
servo_remote.connect()

连接到ESP32 (192.168.3.132:8888)...
成功连接到 192.168.3.132:8888


True

In [ ]:
servo_local = ST3215(port='/dev/ttyACM0')  
INTERVAL = 300
ts = time.time()
count = 0
try:
    while True:
        if time.time() - ts > INTERVAL: break 
        count += 1
        #clear_output(wait=True)
        result = servo_local.readPosi(dev_id=m.dev_id, debug=True)
        posi_vec, velo_vec = m.getPosiVelo(result) 
        try:
            servo_remote.send_command("setPosition", dev_id_vec=m.dev_id, posi_vec=posi_vec, velo_vec=velo_vec, silent=False)
        
        except socket.timeout:
            print("DEBUG: Socket timeout - no response received")
            servo_remote = ESP32SERVO(esp32_ip)
            #servo_remote.connect() 
        except Exception as e:
            print(f"服务器未响应: {e}") 
            
except KeyboardInterrupt:
    pass

print(f'{count / INTERVAL}fps')